# Building Basic Chatbots with LangGraph

In this notebook, I'll walk through building simple conversational chatbots using LangGraph. This is where I started getting comfortable with the core concepts:

- **MessagesState**: Built-in state for conversation management
- **Simple Architecture**: Minimal setup for chatbot functionality
- **LLM Integration**: Direct integration with language models
- **Conversation Flow**: Basic request-response patterns
- **Stateless Design**: Each interaction is independent

## What I'm Building

I'll create a basic chatbot that:
1. **Processes User Messages**: Handles text input from users
2. **Generates Responses**: Uses LLM to create appropriate replies
3. **Maintains Context**: Keeps track of the current conversation
4. **Simple Flow**: Straightforward request-response pattern

This approach works well for:
- **Learning LangGraph**: Understanding core concepts
- **Prototyping**: Quick chatbot development
- **Simple Use Cases**: FAQ bots, basic assistants
- **Foundation Building**: Base for more complex applications

## 1. Essential Imports

Starting with the core components I need for a basic chatbot:

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from IPython.display import display, Image
import os

## 2. State Definition for Conversations

MessagesState is what I use as the foundation for conversational applications in LangGraph.

### Why I chose MessagesState:
- **Built-in Support**: Designed specifically for conversations
- **Message Handling**: Automatic management of conversation flow
- **Type Safety**: Ensures proper message structure
- **LangChain Integration**: Works seamlessly with LangChain components

### Key Components:
- **messages**: List of conversation messages
- **add_messages**: Function to append new messages
- **Annotated**: Type hint for proper message accumulation

### Other approaches I considered:
- **Custom State**: More control but requires manual message handling
- **Simple Dictionary**: Less structure but more flexibility
- **Extended MessagesState**: Add custom fields to the base state

In [ ]:
class ChatState(TypedDict):
    """State definition for my basic chatbot.
    
    This state manages the conversation messages using LangGraph's
    built-in message handling capabilities.
    
    The Annotated type with add_messages ensures that new messages
    are properly appended to the conversation history.
    """
    messages: Annotated[list, add_messages]

## 3. LLM Configuration

I'm using AWS Bedrock with Claude for my language model.

### Model Selection - what I learned:
- **Claude Sonnet**: Balanced performance and cost
- **Claude Haiku**: Faster responses, lower cost
- **Claude Opus**: Highest quality for complex tasks

### Environment Setup:
Make sure your `.env` file contains:
```
AWS_ACCESS_KEY_ID=your_access_key
AWS_SECRET_ACCESS_KEY=your_secret_key
```

### Other LLM options I could use:
- **OpenAI**: `ChatOpenAI` for GPT models
- **Anthropic**: Direct `ChatAnthropic` integration
- **Local Models**: `Ollama` for on-premise deployment
- **Azure**: `AzureChatOpenAI` for Microsoft cloud

In [ ]:
from dotenv import load_dotenv
from langchain_aws import ChatBedrock

load_dotenv()

def get_chat_model():
    """Initialize and return a ChatBedrock model instance.
    
    This function creates a connection to AWS Bedrock with Claude Sonnet,
    which provides a good balance of capability and performance for
    conversational applications.
    
    Returns:
        ChatBedrock: Configured language model for conversations
    """
    llm = ChatBedrock(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
        region="us-east-1"
    )
    return llm

print("LLM configuration ready")

## 4. Prompt Engineering for Chatbots

Prompt design is something I had to get right for chatbot behavior and personality.

### What I learned about prompt design:
- **Clear Role Definition**: Establish what the chatbot is
- **Behavior Guidelines**: How it should respond
- **Tone and Style**: Personality characteristics
- **Conversation Context**: How to handle message history

### System Prompt Components:
1. **Identity**: Who/what the chatbot is
2. **Capabilities**: What it can and cannot do
3. **Behavior**: How it should interact
4. **Constraints**: Important limitations or guidelines

### What works in practice:
- **Be Specific**: Clear instructions prevent confusion
- **Set Expectations**: Define the chatbot's role clearly
- **Include Examples**: Show desired behavior when needed
- **Handle Edge Cases**: Address common problematic scenarios

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

def create_chatbot_chain():
    """Create a conversation chain for my basic chatbot.
    
    This chain combines a system prompt that defines the chatbot's
    personality and behavior with a placeholder for conversation messages.
    
    Returns:
        Chain: LangChain chain for processing conversations
    """
    llm = get_chat_model()
    
    # Define the chatbot's personality and behavior
    system_prompt = (
        "You are a helpful and friendly chatbot assistant. "
        "Your goal is to be helpful, informative, and engaging in conversations. "
        "You should:"
        "\n- Provide clear and accurate information when possible"
        "\n- Ask clarifying questions when needed"
        "\n- Be conversational and maintain a friendly tone"
        "\n- Acknowledge when you don't know something"
        "\n- Keep responses concise but thorough"
        "\n- Remember the context of the conversation"
    )
    
    # Create the prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages")
    ])
    
    # Combine prompt and LLM into a chain
    chain = prompt | llm
    
    return chain

print("Chatbot chain created")

## 5. Chatbot Node Implementation

The chatbot node is the core processing unit that handles user messages.

### What this node does:
1. **Receive State**: Get current conversation state
2. **Process Messages**: Use the LLM chain to generate responses
3. **Return Updates**: Provide new messages to update the state

### Critical things I learned:
- **Input**: Must accept the full state as parameter
- **Processing**: Use the conversation chain with message history
- **Output**: Return dictionary with state updates
- **Message Handling**: New messages are automatically appended

### Error handling I had to add:
- **LLM Failures**: Handle API errors gracefully
- **Invalid Input**: Manage malformed messages
- **Rate Limits**: Handle API rate limiting
- **Timeouts**: Manage slow responses

In [ ]:
def chatbot_node(state: ChatState) -> ChatState:
    """Process user messages and generate chatbot responses.
    
    This node takes the current conversation state, processes it through
    our chatbot chain, and returns the LLM's response as a new message.
    
    The add_messages annotation in our state definition ensures that
    the response is appended to the conversation history rather than
    replacing the existing messages.
    
    Args:
        state: Current conversation state with message history
        
    Returns:
        State update containing the chatbot's response message
    """
    try:
        # Get the conversation chain
        chain = create_chatbot_chain()
        
        # Process the conversation through the chain
        response = chain.invoke({"messages": state["messages"]})
        
        # Return the response as a state update
        return {"messages": response}
        
    except Exception as e:
        # Handle errors gracefully
        print(f"Error in chatbot_node: {e}")
        
        # Return an error message
        from langchain_core.messages import AIMessage
        error_response = AIMessage(
            content="I apologize, but I encountered an error processing your message. Please try again."
        )
        return {"messages": error_response}

print("Chatbot node implemented")

## 6. Building the Chatbot Graph

My chatbot uses a simple linear graph structure:

```
START → chatbot_node → END
```

### Why I chose this design:
- **Simplicity**: Single node handles all conversation logic
- **Clarity**: Easy to understand and debug
- **Extensibility**: Simple to add more nodes later
- **Performance**: Minimal overhead for basic conversations

### Why this architecture works:
- **Single Responsibility**: One node, one purpose
- **State Management**: LangGraph handles message accumulation
- **Flexibility**: Easy to modify behavior by changing the node
- **Scalability**: Can handle multiple concurrent conversations

### Extensions I could add later:
- **Intent Classification**: Add routing based on user intent
- **Tool Integration**: Add function calling capabilities
- **Content Filtering**: Add moderation and safety checks
- **Analytics**: Add logging and conversation tracking

In [ ]:
def create_chatbot_graph():
    """Build and compile the chatbot graph.
    
    This function creates a simple linear graph with a single chatbot node
    that processes all user messages and generates appropriate responses.
    
    Returns:
        Compiled graph ready for conversation processing
    """
    # Initialize the graph with our state type
    graph = StateGraph(ChatState)
    
    # Add the chatbot processing node
    graph.add_node("chatbot", chatbot_node)
    
    # Set up the flow: START → chatbot → END
    graph.add_edge(START, "chatbot")
    graph.add_edge("chatbot", END)
    
    # Compile the graph into an executable application
    app = graph.compile()
    
    return app

# Create our chatbot application
chatbot_app = create_chatbot_graph()

print("Chatbot graph compiled successfully")

## 7. Graph Visualization

Let me visualize this simple but effective chatbot architecture:

In [ ]:
try:
    display(Image(chatbot_app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Visualization error: {e}")
    print("\nGraph Structure:")
    print("START → chatbot_node → END")
    print("\nThis simple linear flow processes user messages and generates responses.")

## 8. Testing the Basic Chatbot

Let me test this chatbot with various types of messages to see how it performs.

### My test strategy:
1. **Simple Greeting**: Basic conversation starter
2. **Information Request**: Ask for specific information
3. **Follow-up Question**: Test conversation context
4. **Complex Query**: More challenging request

### What I'm looking for:
- **Response Quality**: Appropriate and helpful responses
- **Context Awareness**: References to previous messages
- **Personality**: Consistent friendly and helpful tone
- **Error Handling**: Graceful handling of issues

In [ ]:
from langchain_core.messages import HumanMessage

# Test 1: Simple greeting
print("Test 1: Simple Greeting")
print("=" * 50)

greeting_message = "Hello! How are you today?"
print(f"User: {greeting_message}")

result = chatbot_app.invoke({
    "messages": [HumanMessage(content=greeting_message)]
})

print(f"Chatbot: {result['messages'][-1].content}")
print("\n" + "=" * 50)

In [ ]:
# Test 2: Information request
print("Test 2: Information Request")
print("=" * 50)

info_message = "Can you explain what LangGraph is and how it works?"
print(f"User: {info_message}")

result = chatbot_app.invoke({
    "messages": [HumanMessage(content=info_message)]
})

print(f"Chatbot: {result['messages'][-1].content}")
print("\n" + "=" * 50)

In [ ]:
# Test 3: Multi-turn conversation
print("Test 3: Multi-turn Conversation")
print("=" * 50)

# Start a conversation
conversation_messages = [
    HumanMessage(content="I'm learning about AI and chatbots. Can you help me?")
]

print(f"User: {conversation_messages[0].content}")

# Get first response
result = chatbot_app.invoke({"messages": conversation_messages})
conversation_messages = result["messages"]

print(f"Chatbot: {conversation_messages[-1].content}")
print()

# Continue the conversation
follow_up = "What are the main differences between rule-based and AI-powered chatbots?"
conversation_messages.append(HumanMessage(content=follow_up))

print(f"User: {follow_up}")

# Get second response
result = chatbot_app.invoke({"messages": conversation_messages})
conversation_messages = result["messages"]

print(f"Chatbot: {conversation_messages[-1].content}")
print("\n" + "=" * 50)

## 9. Streaming Responses

For a more interactive experience, let me implement streaming responses.

### Why streaming is useful:
- **Real-time Feedback**: Users see responses as they're generated
- **Better UX**: Reduces perceived latency
- **Interruptible**: Can stop generation if needed
- **Progressive Display**: Show partial responses

### When I use streaming:
- **Long Responses**: When answers might be lengthy
- **Interactive Applications**: Real-time chat interfaces
- **User Experience**: When responsiveness is important
- **Live Demos**: Showing the AI "thinking" process

In [ ]:
# Test 4: Streaming response
print("Test 4: Streaming Response")
print("=" * 50)

streaming_message = "Tell me a story about a robot learning to be creative."
print(f"User: {streaming_message}")
print("\nChatbot (streaming): ", end="")

# Use streaming to show response as it's generated
for chunk in chatbot_app.stream({
    "messages": [HumanMessage(content=streaming_message)]
}):
    # Extract the message content from the chunk
    if "chatbot" in chunk:
        message = chunk["chatbot"]["messages"]
        if hasattr(message, 'content'):
            print(message.content)
        else:
            print(message)

print("\n" + "=" * 50)

## 10. Interactive Chat Session

Let me create a simple interactive chat interface to demonstrate the full chatbot experience.

### What this interface provides:
- **Continuous Conversation**: Multiple exchanges in one session
- **Context Preservation**: Maintains conversation history
- **User Control**: Easy exit mechanism
- **Error Handling**: Graceful error recovery

### How to use it:
1. Type your message and press Enter
2. The chatbot will respond based on the conversation history
3. Type 'quit', 'exit', or 'bye' to end the conversation
4. The conversation context is maintained throughout the session

In [ ]:
def interactive_chat():
    """Run an interactive chat session with the chatbot.
    
    This function provides a simple command-line interface for
    chatting with our LangGraph-based chatbot.
    """
    print("Interactive Chatbot Session")
    print("=" * 50)
    print("Type your messages and press Enter to chat.")
    print("Type 'quit', 'exit', or 'bye' to end the conversation.")
    print("=" * 50)
    
    # Initialize conversation history
    conversation_messages = []
    
    while True:
        try:
            # Get user input
            user_input = input("\nYou: ").strip()
            
            # Check for exit conditions
            if user_input.lower() in ['quit', 'exit', 'bye', 'goodbye']:
                print("\nGoodbye! Thanks for chatting!")
                break
            
            # Skip empty inputs
            if not user_input:
                print("Please enter a message or type 'quit' to exit.")
                continue
            
            # Add user message to conversation
            conversation_messages.append(HumanMessage(content=user_input))
            
            # Get chatbot response
            print("\nChatbot: ", end="")
            
            result = chatbot_app.invoke({"messages": conversation_messages})
            
            # Update conversation with the response
            conversation_messages = result["messages"]
            
            # Display the response
            print(conversation_messages[-1].content)
            
        except KeyboardInterrupt:
            print("\n\nChat interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"\nError: {e}")
            print("Please try again or type 'quit' to exit.")

# Run the interactive chat
interactive_chat()

## My Key Takeaways

### What I learned building this:

1. **Basic Chatbot Architecture**: Simple linear graph for conversation processing
2. **MessagesState**: Built-in state management for conversations
3. **LLM Integration**: Direct integration with language models
4. **Prompt Engineering**: Designing effective system prompts for chatbots
5. **Error Handling**: Graceful handling of failures and edge cases
6. **Interactive Interfaces**: Building user-friendly chat experiences

### When this pattern works well:

- **Simple Chatbots**: Basic question-answering applications
- **Prototyping**: Quick development and testing of conversational AI
- **Learning**: Understanding LangGraph fundamentals
- **Foundation**: Base for more complex conversational applications

### Good practices I discovered:

- **State Management**: Using TypedDict and add_messages for proper conversation flow
- **Error Handling**: Graceful degradation when things go wrong
- **Modular Design**: Separate functions for different responsibilities
- **User Experience**: Clear interfaces and helpful feedback
- **Testing Strategy**: Comprehensive testing of different scenarios

### Where this approach falls short:

- **No Memory**: Conversations don't persist between sessions
- **No Context**: Each invocation is independent
- **Limited Capabilities**: No tool use or external integrations
- **Simple Routing**: No intent classification or conditional logic

### What I'm building next:

- **Notebook 4**: Add memory capabilities for persistent conversations
- **Notebook 5**: Implement SQLite-based persistent storage
- **Advanced Features**: Tool integration, function calling, and external APIs
- **Production Deployment**: Scaling and monitoring considerations

### Extensions I could add:

- **Intent Classification**: Route different types of queries to specialized handlers
- **Tool Integration**: Add function calling for external data access
- **Content Moderation**: Filter inappropriate content and responses
- **Analytics**: Track conversation metrics and user satisfaction
- **Personalization**: Adapt responses based on user preferences

### Architecture patterns I've explored:

- **Linear Flow**: Simple request-response (this notebook)
- **Conditional Routing**: Branch based on user intent (Notebook 2)
- **Memory-Enhanced**: Persistent conversation history (Notebook 4)
- **Tool-Augmented**: External function calling capabilities
- **Multi-Agent**: Coordination between multiple AI agents

### Production considerations I need to think about:

- **Scalability**: Consider load balancing for multiple users
- **Security**: Implement authentication and input validation
- **Monitoring**: Add logging and performance metrics
- **Rate Limiting**: Prevent abuse and manage costs
- **Error Recovery**: Implement retry logic and fallback responses